<a href="https://colab.research.google.com/github/torquerxf/Delivery-Duration-Prediction/blob/main/doordash_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Steps Breakdown

- Data Preprocessing
- Feature Engineering
- Encoding & Scaling
- Model Selection
- Evaluation
- Model Interpretation

### Data Preprocessing

In [1]:
!git clone https://github.com/torquerxf/delivery-duration-prediction.git
%cd delivery-duration-prediction

!unzip datasets.zip -d extracted

import pandas as pd
df = pd.read_csv("extracted/datasets/historical_data.csv")

Cloning into 'delivery-duration-prediction'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 18 (delta 6), reused 3 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (18/18), 5.19 MiB | 36.69 MiB/s, done.
Resolving deltas: 100% (6/6), done.
/content/delivery-duration-prediction
Archive:  datasets.zip
   creating: extracted/datasets/
  inflating: extracted/datasets/historical_data.csv  


In [2]:
df.head()

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration
0,1.0,2015-02-06 22:24:17,2015-02-06 23:27:16,1845,american,1.0,4,3441,4,557,1239,33.0,14.0,21.0,446,861.0
1,2.0,2015-02-10 21:49:25,2015-02-10 22:56:29,5477,mexican,2.0,1,1900,1,1400,1400,1.0,2.0,2.0,446,690.0
2,3.0,2015-01-22 20:39:28,2015-01-22 21:09:09,5477,NaN,1.0,1,1900,1,1900,1900,1.0,0.0,0.0,446,690.0
3,3.0,2015-02-03 21:21:45,2015-02-03 22:13:00,5477,NaN,1.0,6,6900,5,600,1800,1.0,1.0,2.0,446,289.0
4,3.0,2015-02-15 02:40:36,2015-02-15 03:20:26,5477,NaN,1.0,3,3900,3,1100,1600,6.0,6.0,9.0,446,650.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197428 entries, 0 to 197427
Data columns (total 16 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   market_id                                     196441 non-null  float64
 1   created_at                                    197428 non-null  object 
 2   actual_delivery_time                          197421 non-null  object 
 3   store_id                                      197428 non-null  int64  
 4   store_primary_category                        192668 non-null  object 
 5   order_protocol                                196433 non-null  float64
 6   total_items                                   197428 non-null  int64  
 7   subtotal                                      197428 non-null  int64  
 8   num_distinct_items                            197428 non-null  int64  
 9   min_item_price                                19

In [4]:
df.isnull().sum()

,0
market_id,987
created_at,0
actual_delivery_time,7
store_id,0
store_primary_category,4760
order_protocol,995
total_items,0
subtotal,0
num_distinct_items,0
min_item_price,0


In [5]:
qual_vars = ['market_id', 'store_id', 'store_primary_category', 'order_protocol']
df[qual_vars].nunique()

,0
market_id,6
store_id,6743
store_primary_category,74
order_protocol,7


Quick Notes:
- market_id, store_id, store_primary_category, order_protocol seem to categorical variables
- created_at and actual_delivery_time have object dtype ---> need to fix it to datatime


In [6]:
data = df.copy() # a copy of the original
# convert the dates to datetime format
data['created_at'] = pd.to_datetime(data['created_at'])
data['actual_delivery_time'] = pd.to_datetime(data['actual_delivery_time'])
# calculate delivery_duration
data['delivery_duration'] = (data['actual_delivery_time'] - data['created_at']).dt.total_seconds()

In [7]:
# drop the not-so relevant features
data.drop(columns=['created_at', 'actual_delivery_time'], inplace=True)

Quick Notes:
- store_primary_category null values can be imputed with a map which maps store_id to its most frequent category

In [8]:
store_id_list = data['store_id'].unique()

store_primary_category_map = {store_id : data[data['store_id'] == store_id]['store_primary_category'].mode() for store_id in store_id_list}



In [9]:
import numpy as np
def fill(store_id):
  try:
    return store_primary_category_map[store_id][0]
  except:
    return np.nan

data['nan_store_primary_category'] = data['store_id'].apply(fill)

In [10]:
data['nan_store_primary_category'].isnull().sum()

np.int64(867)

### Feature Engineering

In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197428 entries, 0 to 197427
Data columns (total 16 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   market_id                                     196441 non-null  float64
 1   store_id                                      197428 non-null  int64  
 2   store_primary_category                        192668 non-null  object 
 3   order_protocol                                196433 non-null  float64
 4   total_items                                   197428 non-null  int64  
 5   subtotal                                      197428 non-null  int64  
 6   num_distinct_items                            197428 non-null  int64  
 7   min_item_price                                197428 non-null  int64  
 8   max_item_price                                197428 non-null  int64  
 9   total_onshift_dashers                         18

Quick Notes:
- dasher_availability_ration = total_busy_dashers / total_onshift_dashers
- the above ratio can contain infinte values! --> fix with adding 1 to denominator
- can combine estimated_order_place_duration and estimated_store_to_consumer_driving_duration as a non_prep_duration  --> (total_processing_time = prep_duration + non_prep_duration)
- price_range = max_item_price - min_item_price
- avg_item_price = subtotal / total_items

In [27]:
data['dasher_availability_ratio'] = data['total_busy_dashers'] / (data['total_onshift_dashers']+1)

In [28]:
data['non_prep_duration'] = data['estimated_order_place_duration'] + data['estimated_store_to_consumer_driving_duration']

In [30]:
# drop old features
data.drop(columns=['total_busy_dashers', 'total_onshift_dashers', 'estimated_order_place_duration', 'estimated_store_to_consumer_driving_duration'], inplace=True)

In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197428 entries, 0 to 197427
Data columns (total 14 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   market_id                   196441 non-null  float64
 1   store_id                    197428 non-null  int64  
 2   store_primary_category      192668 non-null  object 
 3   order_protocol              196433 non-null  float64
 4   total_items                 197428 non-null  int64  
 5   subtotal                    197428 non-null  int64  
 6   num_distinct_items          197428 non-null  int64  
 7   min_item_price              197428 non-null  int64  
 8   max_item_price              197428 non-null  int64  
 9   total_outstanding_orders    181166 non-null  float64
 10  delivery_duration           197421 non-null  float64
 11  nan_store_primary_category  196561 non-null  object 
 12  dasher_availability_ratio   181166 non-null  float64
 13  non_prep_durat

### Encoding & Scaling

Quick Notes:
- will use OneHotEncode for market_id, nan_store_primary_category and order_protocol, reason being they're categorical
- can apply StandardScaler/MinMaxScaler on numerical features

In [32]:
market_id_encoded = pd.get_dummies(data['market_id'], prefix='market_id')

In [37]:
store_primary_category_encoded = pd.get_dummies(data['nan_store_primary_category'], prefix='category')
store_primary_category_encoded.head()

,category_afghan,category_african,category_alcohol,category_alcohol-plus-food,category_american,category_argentine,category_asian,category_barbecue,category_belgian,category_brazilian,...,category_southern,category_spanish,category_steak,category_sushi,category_tapas,category_thai,category_turkish,category_vegan,category_vegetarian,category_vietnamese
0,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [38]:
order_protocol_encoded = pd.get_dummies(data['order_protocol'], prefix='protocol')
order_protocol_encoded.head()

,protocol_1.0,protocol_2.0,protocol_3.0,protocol_4.0,protocol_5.0,protocol_6.0,protocol_7.0
0,True,False,False,False,False,False,False
1,False,True,False,False,False,False,False
2,True,False,False,False,False,False,False
3,True,False,False,False,False,False,False
4,True,False,False,False,False,False,False


In [47]:
data_encoded = pd.concat([data.drop(columns=['market_id', 'store_primary_category', 'nan_store_primary_category', 'order_protocol'], axis=1), market_id_encoded, store_primary_category_encoded, order_protocol_encoded],  axis=1)

### Model Selection

### Evaluation

### Model Interpretation